<a href="https://colab.research.google.com/github/rgvananth/Pytorch_ML/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [0]:
# HyperParametrs
train_batch_size = 100
test_batch_size = 100
num_classes = 10
input_size = 28
learning_rate = 0.001
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_epochs = 10
hidden_size = 150
num_layers = 2
seq_length = 28

In [0]:
# Data Set Initialization and Data Loader

train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           download=True, 
                                           transform=transforms.ToTensor())
test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train = False, 
                                          download=True, 
                                          transform=transforms.ToTensor())

train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                                batch_size=train_batch_size, 
                                                shuffle=True)

test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                                batch_size=test_batch_size, 
                                                shuffle=True)

In [0]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.num_classes = num_classes
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, num_classes)
    
  def forward(self, x):
    # Initial hidden and cell states
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    out, _ = self.lstm(x, (h0,c0))
    #Decode hidden state - last time step
    out = self.fc(out[:, -1, :])
    return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [22]:
for epoch in range(num_epochs):
  for i, (images, label) in enumerate(train_data_loader):
    inputs = images.reshape(-1, seq_length, input_size).to(device)
    label = label.to(device)

    pred = model(inputs)
    loss = criterion(pred, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print('Loss for batch: {} in Epoch : {} is: {}'.format(i+1, epoch, loss))

Loss for batch: 100 in Epoch : 0 is: 0.6752113103866577
Loss for batch: 200 in Epoch : 0 is: 0.5097470283508301
Loss for batch: 300 in Epoch : 0 is: 0.2885512411594391
Loss for batch: 400 in Epoch : 0 is: 0.1788935512304306
Loss for batch: 500 in Epoch : 0 is: 0.13708044588565826
Loss for batch: 600 in Epoch : 0 is: 0.10348205268383026
Loss for batch: 100 in Epoch : 1 is: 0.1789693385362625
Loss for batch: 200 in Epoch : 1 is: 0.05596165359020233
Loss for batch: 300 in Epoch : 1 is: 0.05727647244930267
Loss for batch: 400 in Epoch : 1 is: 0.02008962072432041
Loss for batch: 500 in Epoch : 1 is: 0.22185809910297394
Loss for batch: 600 in Epoch : 1 is: 0.11218936741352081
Loss for batch: 100 in Epoch : 2 is: 0.03229854255914688
Loss for batch: 200 in Epoch : 2 is: 0.12151230871677399
Loss for batch: 300 in Epoch : 2 is: 0.05640658363699913
Loss for batch: 400 in Epoch : 2 is: 0.04344066604971886
Loss for batch: 500 in Epoch : 2 is: 0.022698871791362762
Loss for batch: 600 in Epoch : 2 is

In [24]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_data_loader:
        images = images.reshape(-1, seq_length, input_size).to(device)
        outputs = model(images).cpu()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 98 %
